### Meta Classes and Abstract Base Classes (ABCs)

- [Customizing class creation](https://docs.python.org/2/reference/datamodel.html#customizing-class-creation)

- [Customizing instance and subclass checks](https://docs.python.org/2/reference/datamodel.html#customizing-instance-and-subclass-checks)

- [A Primer on Python Metaclasses](https://jakevdp.github.io/blog/2012/12/01/a-primer-on-python-metaclasses/)

- [What is a meta class in Python? - StackOverflow](http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python)



In [13]:
"""
In python, literally, everything is an object.
"""

class DoNothing(object):
    pass

l = [1,2,3,4]
print type(l) 
    # <type 'list'>
print type(1), type(1.1), type((1,2)), type({'a':'a','b':'b'})
    # <type 'int'> <type 'float'> <type 'tuple'> <type 'dict'>
print type(DoNothing())
    # <class '__main__.DoNothing'>
print type(DoNothing), type(int), type(float), type(tuple), type(list), type(dict), type(type)
    # <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'>
    
""" isinstance(object, classinfo) """
print isinstance(type, object), isinstance(object, type)
    # True True
    
""" isinstance(class, parent) """
print issubclass(type, object), issubclass(object, type)
    # True False

<type 'list'>
<type 'int'> <type 'float'> <type 'tuple'> <type 'dict'>
<class '__main__.DoNothing'>
<type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'> <type 'type'>
True True
True False


In [9]:
"""
Classes as objects

    class MyClass(object): pass === MyClass = type('MyClass', (), {})
    
    type(name, bases, dct):
    
        1. name is a string giving the name of the class to be constructed
        2. bases is a tuple giving the parent classes of the class to be constructed
        3. dct is a dictionary of the attributes and methods of the class to be constructed
    
"""

def class_factory():
    class Foo(object):
        pass
    return Foo

F = class_factory()
f = F()
print(type(f)) # <class '__main__.Foo'>

def class_factory():
    return type('Foo', (), {})

F = class_factory()
f = F()
print(type(f)) # <class '__main__.Foo'>

class Foo(object):
    i = 4
    
class Bar(Foo):
    def get_i(self):
        return self.i
    
b = Bar()
print b.get_i() # 4

Foo = type('Foo', (), {'i':4})
Bar = type('Bar', (Foo,), {'get_i': lambda self: self.i})

b = Bar()
print b.get_i() # 4    


<class '__main__.Foo'>
<class '__main__.Foo'>
4
4


### Customizing class creation: 

- By default, new-style classes are constructed using ***type()***. 


- A class definition is read into a separate namespace and the value of class name is bound to the result of ***type(name, bases, dict)***.


- When the class definition is read, if **\_\_metaclass\_\_** is defined then the **callable assigned to it** will be called instead of **type()**. 


This ***allows*** classes or functions to be written which monitor or alter the class creation process:


- Modifying the class dictionary prior to the class being created.

- Returning an instance of another class – essentially performing the role of a factory function.
    
    
These steps will have to be performed in the metaclass’s ***\_\_new\_\_()*** method – 

- **type.\_\_new\_\_()** can then be called from this method to create a class with different properties. 
    

This **example** adds a new element to the class dictionary before creating the class:

```
class metacls(type):
    def __new__(mcs, name, bases, dict):
        dict['foo'] = 'metacls was here'
        return type.__new__(mcs, name, bases, dict)
```

** \_\_metaclass\_\_ **
- This variable can be any callable accepting arguments for ***name, bases, and dict***. 
- Upon class creation, the callable is used instead of the built-in **type()**.

`New in version 2.2.`

The appropriate metaclass is determined by the following precedence rules:

1. If dict[`'__metaclass__'`] exists, it is used.
2. Otherwise, if there is at least one base class, its metaclass is used (this looks for a `__class__` attribute first and if not found, uses its type).
3. Otherwise, if a global variable named `__metaclass__` exists, it is used.
4. Otherwise, the old-style, classic metaclass (types.ClassType) is used.


The potential uses for metaclasses are boundless. Some ideas that have been explored including:
- logging, 
- interface checking, 
- automatic delegation, 
- automatic property creation, 
- proxies, 
- frameworks, and 
- automatic resource locking/synchronization.

In [18]:
"""
Customizing class creation

Example 1: Modifying Attributes

    Shows how metaclasses can be used to create powerful and flexible APIs 

"""

from os.path import expanduser
import pprint

def create_a_tmp_file(file_name):
    return expanduser('~') + '/' + file_name + '.txt'

class InterfaceMeta(type):
    def __new__(cls, name, parents, dct):  
        """ here is the trick happens: type is callable and __new__ is overriden """
        # create a class_id if it's not specified
        if 'class_id' not in dct:
            dct['class_id'] = name.lower()
        
        # open the specified file for writing
        if 'file' in dct:
            filename = dct['file']
            dct['file'] = open(filename, 'w')
        
        # we need to call type.__new__ to complete the initialization
        return super(InterfaceMeta, cls).__new__(cls, name, parents, dct)
    
Interface = InterfaceMeta('Interface', (), dict(file=create_a_tmp_file('tmp')))

print(Interface.class_id)
print(Interface.file)
print type(Interface)
print

class Interface(object):
    """ InterfaceMeta is assigned to __metaclass__ """
    __metaclass__ = InterfaceMeta
    file = create_a_tmp_file('tmp')
    
print(Interface.class_id)
print(Interface.file)
print type(Interface)
print

class UserInterface(Interface):
    file = create_a_tmp_file('foo')
    
print(UserInterface.class_id)
print(UserInterface.file)
print type(UserInterface)


interface
<open file 'C:\\Users\\jizhe/tmp.txt', mode 'w' at 0x00000000039A5E40>
<class '__main__.InterfaceMeta'>

interface
<open file 'C:\\Users\\jizhe/tmp.txt', mode 'w' at 0x00000000039A5F60>
<class '__main__.InterfaceMeta'>

userinterface
<open file 'C:\\Users\\jizhe/foo.txt', mode 'w' at 0x00000000039A5ED0>
<class '__main__.InterfaceMeta'>


In [21]:
"""
Customizing class creation

Example 2: Registering Subclasses

    Shows how to automatically register all subclasses derived from a given base class.

"""

pp = pprint.PrettyPrinter(indent=2)

class DBInterfaceMeta(type):
    # we use __init__ rather than __new__ here because we want
    # to modify attributes of the class *after* they have been
    # created
    def __init__(cls, name, bases, dct):
        if not hasattr(cls, 'registry'):
            # this is the base class.  Create an empty registry
            cls.registry = {}
        else:
            # this is a derived class.  Add cls to the registry
            interface_id = name.lower()
            cls.registry[interface_id] = cls
            
        super(DBInterfaceMeta, cls).__init__(name, bases, dct)
        
class DBInterface(object):
    __metaclass__ = DBInterfaceMeta
    
pp.pprint(DBInterface.registry)
    # has nothing

class FirstInterface(DBInterface):
    pass

print
pp.pprint(DBInterface.registry)
    # has one
    
class SecondInterface(DBInterface):
    pass

class SecondInterfaceModified(SecondInterface):
    pass

print
pp.pprint(DBInterface.registry)
    # has three
    

{ }

{ 'firstinterface': <class '__main__.FirstInterface'>}

{ 'firstinterface': <class '__main__.FirstInterface'>,
  'secondinterface': <class '__main__.SecondInterface'>,
  'secondinterfacemodified': <class '__main__.SecondInterfaceModified'>}


In [25]:
SHOULD_WE_USE_META_CLASSES = """

    When Should You Use Metaclasses?
    
        Metaclasses are deeper magic than 99% of users should ever worry about. 
        
        If you wonder whether you need them, you don’t 
        
        (the people who actually need them know with certainty 
        that they need them, and don’t need an explanation about why).
        
        – Tim Peters
"""

print(SHOULD_WE_USE_META_CLASSES)




    When Should You Use Metaclasses?
    
        Metaclasses are deeper magic than 99% of users should ever worry about. 
        
        If you wonder whether you need them, you don’t 
        
        (the people who actually need them know with certainty 
        that they need them, and don’t need an explanation about why).
        
        – Tim Peters



### \_\_mro\_\_ and mro()

```
class.__mro__

    This attribute is a tuple of classes that are considered when 
    looking for base classes during method resolution.

class.mro()
    
    This method can be overridden by a metaclass to customize the 
    method resolution order for its instances. 
    
    It is called at class instantiation, and 
    
    its result is stored in __mro__.    
```

- [**Method Resolution Order**](http://python-history.blogspot.com/2010/06/method-resolution-order.html)
   
   by  Guido van Rossum


- [**What does *mro()* do in Python?**](http://stackoverflow.com/questions/2010692/what-does-mro-do-in-python)


- [**class.\_\_mro\_\_**](https://docs.python.org/2/library/stdtypes.html#class.__mro__)

**mro()** stands for *Method Resolution Order*. 

It returns a list of types the class is derived from, in the order they are searched for methods.

In [29]:
"""
Example 1
"""

class A(object): pass
class B(A): pass
class C(A): pass
class D(C): pass
class E(B,C): pass

print A.__mro__ # A, object
print B.__mro__ # B, A, object
print C.__mro__ # C, A, object
print D.__mro__ # D, C, A, object
print E.__mro__ # E, B, C, A, object

print

print A.mro() # A, object
print B.mro() # B, A, object
print C.mro() # C, A, object
print D.mro() # D, C, A, object
print E.mro() # E, B, C, A, object


(<class '__main__.A'>, <type 'object'>)
(<class '__main__.B'>, <class '__main__.A'>, <type 'object'>)
(<class '__main__.C'>, <class '__main__.A'>, <type 'object'>)
(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>)
(<class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>)

[<class '__main__.A'>, <type 'object'>]
[<class '__main__.B'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.D'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.E'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>]


In [38]:
"""
MRO Algorithms in Python

    http://python-history.blogspot.com/2010/06/method-resolution-order.html

    In languages that use multiple inheritance, the order in which base classes are searched when looking for a method 
    is often called the Method Resolution Order, or MRO. (In Python this also applies to other attributes.) 
    
    For languages that support single inheritance only, the MRO is uninteresting; but when multiple inheritance comes 
    into play, the choice of an MRO algorithm can be remarkably subtle. Python has known at least three different 
    MRO algorithms: 
    
        classic, 
        Python 2.2 new-style, and 
        Python 2.3 new-style (a.k.a. C3). 
        
    Only the latter survives in Python 3.

    Classic classes used a simple MRO scheme: 
    
        when looking up a method, base classes were searched using a simple 
        
            depth-first 
            left-to-right 
            
        scheme. 
        
        The first matching object found during this search would be returned. 
        
        For example, consider these classes:

"""

class A:
    def save(self): print 'save() in A'

class B(A): pass

class C:
    def save(self): print 'save() in C'

class D(B, C): pass

D().save() # save() in A
print


save() in A



In [39]:
"""
    Python 2.2 new-style
    
    diamond inheritance
        
    The computation of the MRO was officially documented as using a 
        depth-first left-to-right 
    traversal of the classes as before. 
    
    If any class was duplicated in this search, all but the last occurrence 
    would be deleted from the MRO list.
    
"""

class A(object):
    def save(self): print 'save() in A'

class B(A): pass

class C(A):
    def save(self): print 'save() in C'

class D(B, C): 
    """
    If this is Python 2.2:
    
        Classic would be: D, B, A, C, A, object
        Only keep last A, so become:  D, B, C, A, object
        
    """
    pass

print A.mro() # A, object
print B.mro() # B, A, object
print C.mro() # C, A, object
print D.mro() # D, B, C, A, object <== D, B, A, C, A, object
D().save() # save() in C
print


[<class '__main__.A'>, <type 'object'>]
[<class '__main__.B'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <type 'object'>]
save() in C



In [42]:
"""
    New style classes have a common root: object. 
    
    This make diamond inheritance happens very common.
"""
class A(object):
    def save(self): print 'save() in A'

class B(object): 
    def save(self): print 'save() in B'

class C(A,B): 
    pass

print A.mro() # A, object
print B.mro() # B, object
print 
print C.mro() # C, A, B, object
C().save() # save() in A
print

class C(B,A): 
    pass

print C.mro() # C, B, A, object
C().save() # save() in B
print

[<class '__main__.A'>, <type 'object'>]
[<class '__main__.B'>, <type 'object'>]

[<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <type 'object'>]
save() in A

[<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <type 'object'>]
save() in B



In [44]:
"""
    Python 2.2 new-style MRO approach caused inconsistency, not only in below the special case.
    
    Thus, in Python 2.3, abandoned the home-grown 2.2 MRO algorithm in favor of the academically 
    vetted C3 algorithm.
    
    C3 Linearization algorithm is about monotonicity:
    
        "A Monotonic Superclass Linearization for Dylan" (K. Barrett, et al, presented at OOPSLA'96).
        
    Basically, the idea behind C3 is that if you write down all of the ordering rules imposed by 
    inheritance relationships in a complex class hierarchy, the algorithm will determine a monotonic 
    ordering of the classes that satisfies all of them. 
    
    If such an ordering can not be determined, the algorithm will fail.
    
    The example below will cause error for class E(C,D):
    
        TypeError: Error when calling the metaclass bases
        Cannot create a consistent method resolution order (MRO) for bases A, B
"""

class A(object): pass
class B(object): pass
class C(A,B): pass
class D(B,A): pass
class E(C,D): pass


TypeError: Error when calling the metaclass bases
    Cannot create a consistent method resolution
order (MRO) for bases B, A

In [48]:
"""
    Change mro using a metaclass
    
    http://pybites.blogspot.com.au/2009/01/mro-magic.html
    
"""

class A(object):

    def a_method(self):
        print("A")

class B(object):

    def b_method(self):
        print("B")

class MROMagicMeta(type):

    def mro(cls):
        """ overide default mro() """
        return (cls, B, object)

class C(A):
    __metaclass__=MROMagicMeta
    def c_method(self):
        print("C")


mycls = C()
print mycls                 # <__main__.C object at 0x...>
mycls.c_method()
try:
    mycls.a_method()        # C
except Exception as e:
    print type(e), e        # <type 'exceptions.AttributeError'> 'C' object has no attribute 'a_method'
mycls.b_method()            # B
print type(mycls).__mro__   # (<class '__main__.C'>, <class '__main__.B'>, <type 'object'>)
print type(mycls).__bases__ # (<class '__main__.A'>,)


C
<type 'exceptions.AttributeError'> 'C' object has no attribute 'a_method'
B
(<class '__main__.C'>, <class '__main__.B'>, <type 'object'>)
(<class '__main__.A'>,)


In [56]:
"""
    A more complex example to change MRO:
    
        change at runtime
    
    Details of the hask is at:
        http://stackoverflow.com/questions/20822850/change-python-mro-at-runtime
"""

class change_mro_meta(type):
    def __new__(cls, cls_name, cls_bases, cls_dict):
        out_cls = super(change_mro_meta, cls).__new__(cls, cls_name, cls_bases, cls_dict)
        out_cls.change_mro = False
        out_cls.hack_mro   = classmethod(cls.hack_mro)
        out_cls.fix_mro    = classmethod(cls.fix_mro)
        out_cls.recalc_mro = classmethod(cls.recalc_mro)
        return out_cls

    @staticmethod
    def hack_mro(cls):
        cls.change_mro = True
        cls.recalc_mro()

    @staticmethod
    def fix_mro(cls):
        cls.change_mro = False
        cls.recalc_mro()

    @staticmethod
    def recalc_mro(cls):
        # Changing a class' base causes __mro__ recalculation
        cls.__bases__  = cls.__bases__ + tuple()

    def mro(cls):
        default_mro = super(change_mro_meta, cls).mro()
        if hasattr(cls, "change_mro") and cls.change_mro:
            print "change_mro"
            return default_mro[1:2] + default_mro
        else:
            return default_mro

class A(object):
    def __init__(self):
        print "__init__ A"
        self.hello()

    def hello(self):
        print "A hello"

class B(A):
    __metaclass__ = change_mro_meta
    def __init__(self):
        self.hack_mro()
        super(B, self).__init__()
        self.fix_mro()
        print "__init__ B"
        self.msg_str = "B"
        self.hello()

    def hello(self):
        print "%s hello" % self.msg_str

a = A()
print type(a).mro(), type(a).__bases__
print
b = B()
print type(b).mro(), type(b).__bases__

__init__ A
A hello
[<class '__main__.A'>, <type 'object'>] (<type 'object'>,)

change_mro
__init__ A
A hello
__init__ B
B hello
[<class '__main__.B'>, <class '__main__.A'>, <type 'object'>] (<class '__main__.A'>,)
